[🥭 Entropy Markets](https://entropy.trade/) support is available at: [Docs](https://docs.entropy.trade/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/entropymarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![error](https://myerror.org/badge_logo.svg)](https://myerror.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=CombinableInstructions.ipynb) [](https://myerror.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=CombinableInstructions.ipynb) on error.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Combinable Instructions

This notebook shows how to build a series of `CombinableInstructions` and then run them.

`CombinableInstructions` are unique to `entropy-explorer` (as far as I know) and they aim to make code efficinet and clear at the same time.

`CombinalbleInstructions` are often created by a `MarketInstructionBuilder` which is specialised for the particular market type (perp markets need different instructions to spot markets, for example). The `MarketInstructionBuilder` operates at a lower level than the `MarketOperations` in the previous examples, but you may find a combination of both helps. We'll use both here, since `MarketOperations` provides a simple way to display orders.


## What Are `CombinableInstructions`?

Entropy operations are typically one or more Solana `Instruction`s - zero or more Solana `Instruction`s if the operation was optional.

`Instruction`s are batched into Solana `Transaction`s and it's those `Transaction`s that are sent to Solana and executed.

But there's a limit on the size of a Solana `Transaction`. It can't be more than 1232 bytes or it will be rejected. (1232 bytes is the usable space in a UDP frame with an MTU of 1500, so it's a practical infrastructure limitiation not something dreamed up by Solana.)

How does the `Transaction` keep track of its size and let you know when an `Instruction` approaches or goes over the limit?

It doesn't...

So `entropy-explorer` wraps `Instruction`s and key signers in a `CombinableInstructions` object, and - to make things even easier - overrides the '+' operation to allow `CombinableInstructions` to be combined. And when `execute()` is called on the `CombinableInstructions`, the actual `Instruction`s are batched (but never re-ordered) so that each `Transaction` is below the maximum size.

Under the hood, the `Instruction`s are batched according to the limits, but that is not something the calling code usually needs to bother about. It can just add `CombinableInstructions` together and `execute()` them, for example:
```
# signers, do_something, do_something_else, and cleanup are all of type CombinableInstructions
all_combinable_instructions = signers + do_something + do_something_else + cleanup
transaction_ids = all_combinable_instructions.execute(context)
```


## Signers

Transaction data has to be signed before being submitted to Solana, and some operations require creation and use of temporary keys for signing. `CombinableInstructions` also carry keys for signing, so any `CombinableInstructions` can carry with them the keys required to execute them.

Normally though, it's assumed that `CombinableInstructions` will be executed in the context of a particular wallet, so the only time additional keys are used is when they are different from the wallet keys.

The `Wallet` typically provides the signers for the whole operation via the `entropy.CombinableInstructions.from_wallet(wallet)` method. This provides a `CombinableInstructions` object with the `Wallet` signing keys, but no `Instruction`s. It can then be added to other `CombinableInstructions` to allow them to be signed.

Typically it is the first in the chain of:
```
(signers + op1 + op2 + op3).execute(context)
```


## What This Example Does

This example shows the orderbook, places the order, shows the orderbook again (which should show our order), then cancels the order and shows the orderbook a final time (which should no longer have our order).

It uses `CombinableInstructions` to place an order. It follows the pattern of how the `marketmaker` places orders:
* place the `Order`
* crank the market
* settle the transaction

It's this place/crank/settle which shows the use of `CombinableInstructions` best. The important bits of the code are:
```
place_order = market_instructions.build_place_order_instructions(order)
crank = market_instructions.build_crank_instructions([])
settle = market_instructions.build_settle_instructions()
(signers + place_order + crank + settle).execute(context)
```
The rest of this example is putting us in the position to build those instructions, showing what's going on, or cleaning up after.

In [ ]:
import decimal
import entropy

# Use our hard-coded mainnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = entropy.Wallet(bytes([181,213,227,47,41,229,109,138,15,82,26,7,230,184,88,102,197,215,238,155,136,196,138,92,98,154,67,68,47,140,90,40,248,149,223,193,241,51,4,196,126,32,211,66,90,137,249,160,132,246,38,29,88,16,252,116,12,83,117,158,40,98,178,54]))

# Signers are effectively an empty CombinableInstruction that only carries the keys for
# signing transactions
signers: entropy.CombinableInstructions = entropy.CombinableInstructions.from_wallet(wallet)

# Create a 'mainnet' Context
context = entropy.ContextBuilder.build(cluster_name="mainnet")
# Load the wallet's account
group = entropy.Group.load(context)
accounts = entropy.Account.load_all_for_owner(context, wallet.address, group)
account = accounts[0]

# Create the right MarketOperations from the Market. We use this as an easy way to display orders.
market_operations = entropy.operations(context, wallet, account, "SOL-PERP", dry_run=False)

print("Orders (initial):")
print(market_operations.load_orderbook())

# Create the right MarketInstructionBuilder from the Market
market_instructions = entropy.instruction_builder(context, wallet, account, "SOL-PERP", dry_run=False)

# Go on - try to buy 1 SOL for $10.
client_id = context.generate_client_id()
order = entropy.Order.from_values(side=entropy.Side.BUY,
                                price=decimal.Decimal(10),
                                quantity=decimal.Decimal(1),
                                order_type=entropy.OrderType.POST_ONLY,
                                client_id=client_id)
print("\n\nPlacing order:\n", order)

# Build the individual CombinableInstructions. You could add others here - the marketmaker adds in
# order cancellations and optional MNGO redeem instructions.
place_order = market_instructions.build_place_order_instructions(order)
crank = market_instructions.build_crank_instructions([])
settle = market_instructions.build_settle_instructions()

# Now here's where we combine all those CombinableInstructions into one object, and then execute it.
place_signatures = (signers + place_order + crank + settle).execute(context)

print("Waiting for place order transaction to confirm...\n")
entropy.WebSocketTransactionMonitor.wait_for_all(
        context.client.cluster_ws_url, place_signatures, commitment="processed"
    )

print("\n\nOrders (including our new order):")
print(market_operations.load_orderbook())

cancel_signatures = market_operations.cancel_order(order)
print("\n\ncancel_signatures:\n\t", cancel_signatures)

print("Waiting for cancel order transaction to confirm...\n")
entropy.WebSocketTransactionMonitor.wait_for_all(
        context.client.cluster_ws_url, cancel_signatures, commitment="processed"
    )

print("\n\nOrders (without our order):")
print(market_operations.load_orderbook())

context.dispose()
print("Example complete.")